# Modul 9 Struktur Data: *Heap Tree*, AVL/*Balance Tree*

Kembali ke [Struktur Data (dengan Python)](strukdat2023.qmd)

In [ ]:
import numpy as np
import graphviz as gv

## Implementasi *Heap Tree* dengan *array*

### Implementasi *Max Heap*

### Implementasi *Min Heap*

### *Heapify*: mengubah *binary tree* menjadi *heap tree*

## Implementasi AVL/*Balance Tree* dengan *pointer* (*linked* AVL *tree*)